In [2]:
!pip install beautifulsoup4

In [3]:
!pip install lxml
!pip install html5lib
!pip install requests

import the required packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd # library for data analsysis

setup the url of the website

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

creates a soup object of the type beautifulsoup and reteieves the wikipedia page to the object

In [4]:
soup = BeautifulSoup(source, 'lxml')
tables = soup.find("table")

extract the content from website and store it in pandas dataframe

In [5]:
data = []
columns = []

for index, tr in enumerate(tables.find_all('tr')):
    header = []
    for td in tr.find_all(['th','td']):
        header.append(td.text.strip())
    
    if (index == 0):
        columns = header
    else:
        data.append(header)

df = pd.DataFrame(data = data,columns = columns)

remove any row where there is no value for Borough

In [6]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)

Groupby the neighbourhoods that has the same post code and borough

In [7]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

Any neighbourhood with not assigned value is replaces with it's respective borough name

In [8]:
df['Neighbourhood'].replace("Not assigned", df["Borough"],inplace=True)

get the row count of dataframe

In [17]:
df.shape

(103, 3)

read csv file to lat and longtitudes

In [21]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


set lat and longtitudes based on postcode

In [27]:
coordinates.rename(columns={"Postal Code": "Postcode"}, inplace=True)
coordinates

df_new = df.merge(coordinates, on="Postcode",  how="left")
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
